# Terminations in Microsoft Autogen


it will helpful when we create industry grade projects

why termination matters?

we have used to do some work, say write a story or accomplish some task. when we dont know that how many terms of rotation we will get the answer. so we provide termination condition to stop the agent working.

In [11]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.ollama import OllamaChatCompletionClient

model_client = OllamaChatCompletionClient(model="gemma2:latest")

In [12]:
add_1_agent_first = AssistantAgent(
    name = "add_1_agent_first",
    model_client=model_client,
    system_message="Add 1 to the number and just give out resultant number. Start with 0 if nothing is provided."
)

add_1_agent_second = AssistantAgent(
    name = "add_1_agent_second",
    model_client=model_client,
    system_message="Add 1 to the number and just give out resultant number."
)

add_1_agent_third = AssistantAgent(
    name = "add_1_agent_third",
    model_client=model_client,
    system_message="Add 1 to the number and just give out resultant number."
)

In [13]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third]
)

In [14]:
from autogen_agentchat.ui import Console

await Console(team.run_stream())

Error processing publish message for add_1_agent_first_43272b11-4dc3-47f0-830d-aec1aa87ae2a/43272b11-4dc3-47f0-830d-aec1aa87ae2a
Traceback (most recent call last):
  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: ResponseError: model requires more system memory (7.7 GiB) than is available (7.3 GiB) (status code: 500)
Traceback:
Traceback (most recent call last):

  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_agentchat\teams\_group_chat\_chat_agent_container.py", line 133, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 953, in on_messages_stream
    async for inference_output in self._call_llm(

  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 1107, in _call_llm
    model_result = await model_client.create(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "x:\Autogen\autogen-env\Lib\site-packages\autogen_ext\models\ollama\_ollama_client.py", line 646, in create
    result: ChatResponse = await future
                           ^^^^^^^^^^^^

  Fil

- we need to add termination condition otherwise it will go infinite
- termination condition resets automatically

In [11]:
from autogen_agentchat.conditions import MaxMessageTermination
max_termination  = MaxMessageTermination(5)

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    termination_condition = max_termination
)

In [5]:
from autogen_agentchat.ui import Console

await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1  

---------- TextMessage (add_1_agent_second) ----------
2 

---------- TextMessage (add_1_agent_third) ----------
3 

---------- TextMessage (add_1_agent_first) ----------
4  

---------- TextMessage (add_1_agent_second) ----------
5 



TaskResult(messages=[TextMessage(id='8ffee859-94de-4e61-87ea-43fe48ac9437', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=32, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 9, 20, 0, 210342, tzinfo=datetime.timezone.utc), content='1  \n', type='TextMessage'), TextMessage(id='18101af4-30bb-4fba-8b1f-d88bf4cf041e', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=26, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 9, 20, 1, 568851, tzinfo=datetime.timezone.utc), content='2 \n', type='TextMessage'), TextMessage(id='477ca655-606c-4f4d-9b0e-741b213313bc', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 9, 20, 3, 302084, tzinfo=datetime.timezone.utc), content='3 \n', type='TextMessage'), TextMessage(id='592acd85-8315-4fa9-b237-c442c611dc33', source='add_1_agent_first', models_usage=RequestUsag

In [7]:
agent1 = AssistantAgent(
    name = "story_writer",
    model_client=model_client,
    system_message="Give the story of a brave indian chhatrapati shivaji maharaj, keep it short about 40 words. if critic say 'THE END' anywhere, only output 'THE END'."
)

agent2 = AssistantAgent(
    name = "story_critic",
    model_client=model_client,
    system_message="Continue the story and critic it with feedback. Keep it short and no more than 40 words. If it feels complete , say 'THE END'. ONly Output 'THE END'."
)

In [8]:
from autogen_agentchat.conditions import TextMentionTermination
text_mention_termination = TextMentionTermination("THE END")

teamwithTextTermination = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=text_mention_termination
)

In [9]:
from autogen_agentchat.ui import Console
await Console(teamwithTextTermination.run_stream(task = "Write a story about chhatrapati shivaji maharaj"))

---------- TextMessage (user) ----------
Write a story about chhatrapati shivaji maharaj


---------- TextMessage (story_writer) ----------
Chhatrapati Shivaji Maharaj, the warrior king of Maratha, rose against Mughal oppression.  His guerilla tactics and strategic brilliance liberated his people, establishing an independent Maratha empire. 



---------- TextMessage (story_critic) ----------
He led by example, inspiring loyalty and courage in his army. His reign saw flourishing arts and culture, solidifying the Maratha identity. THE END 



TaskResult(messages=[TextMessage(id='d5e4ca31-5a45-40d9-9ff4-b4613d81c2d7', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 7, 9, 52, 12, 755195, tzinfo=datetime.timezone.utc), content='Write a story about chhatrapati shivaji maharaj', type='TextMessage'), TextMessage(id='25bb4ab0-8bcc-417e-9d76-759ef274134a', source='story_writer', models_usage=RequestUsage(prompt_tokens=63, completion_tokens=42), metadata={}, created_at=datetime.datetime(2025, 10, 7, 9, 52, 25, 458678, tzinfo=datetime.timezone.utc), content='Chhatrapati Shivaji Maharaj, the warrior king of Maratha, rose against Mughal oppression.  His guerilla tactics and strategic brilliance liberated his people, establishing an independent Maratha empire. \n\n\n', type='TextMessage'), TextMessage(id='ffc5cc73-4c10-4334-80c4-f90c8a679c36', source='story_critic', models_usage=RequestUsage(prompt_tokens=102, completion_tokens=33), metadata={}, created_at=datetime.datetime(2025, 10, 7, 9, 52, 41, 8

In [12]:
from autogen_agentchat.conditions import TextMentionTermination


combined_termination = MaxMessageTermination(5) & TextMentionTermination('THE END')

text_mention_termination = TextMentionTermination("THE END")
teamwithTextTermination = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=combined_termination
)


In [ ]:
from autogen_agentchat.ui import Console
await Console(teamwithTextTermination.run_stream(task = "Write a story about chhatrapati shivaji maharaj"))

---------- TextMessage (user) ----------
Write a story about chhatrapati shivaji maharaj
---------- TextMessage (story_writer) ----------
The wind whipped through Shivaji's thick black hair as he stood atop the Pratapgad fort. Below, the sprawling Maratha landscape stretched like a verdant tapestry, kissed by the golden rays of dawn. The young warrior king inhaled deeply, the air crisp with the promise of battle and victory. 

Shivaji wasn't born a king; he was forged in the crucible of adversity. His father, Shahaji Raje Bhosale, a respected Maratha leader, had instilled in him a deep love for his land and a fierce determination to fight for its freedom. He learned warfare from a young age, mastering the art of guerilla tactics under the tutelage of the legendary warrior, Dadaji Konddev.

His rise to power was meteoric. His daring raids on Mughal strongholds, his cunning strategies, and his unwavering courage against insurmountable odds made him a legend in his own time.  He wasn't ju

# External Termination

Enables programmatic control of termination from outside the run. This is useful for UI Integration(e.g "Stop" buttons in chat interfaces).

In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.ollama import OllamaChatCompletionClient

model_client = OllamaChatCompletionClient(model="gemma2:latest")

In [3]:
from autogen_agentchat.agents import AssistantAgent

agent1 = AssistantAgent(
    name = "story_writer",
    model_client=model_client,
    system_message="Give the story of a brave indian chhatrapati shivaji maharaj, keep it short about 40 words. if critic say 'THE END' anywhere, only output 'THE END'."
)

agent2 = AssistantAgent(
    name = "story_critic",
    model_client=model_client,
    system_message="Continue the story and critic it with feedback. Keep it short and no more than 40 words. If it feels complete , say 'THE END'. ONly Output 'THE END'."
)

from autogen_agentchat.conditions import ExternalTermination
external_termination = ExternalTermination()

from autogen_agentchat.teams import RoundRobinGroupChat
team = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=external_termination
)

In [4]:
from autogen_agentchat.ui import Console
run = asyncio.create_task(Console(team.run_stream(task = "Write a story about chhatrapati shivaji maharaj in less than 40 words")))

await asyncio.sleep(2)

external_termination.set()
await run

---------- TextMessage (user) ----------
Write a story about chhatrapati shivaji maharaj in less than 40 words
---------- TextMessage (story_writer) ----------
Chhatrapati Shivaji Maharaj, the Maratha warrior king, fought bravely against Mughal rule, liberating his people and establishing an independent Hindu kingdom.  



TaskResult(messages=[TextMessage(id='55b766ec-94a0-4808-b548-a42a69e5fd83', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 7, 10, 7, 44, 95978, tzinfo=datetime.timezone.utc), content='Write a story about chhatrapati shivaji maharaj in less than 40 words', type='TextMessage'), TextMessage(id='00b92a42-d93f-4286-b2a7-c424cbbe6ce0', source='story_writer', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=33), metadata={}, created_at=datetime.datetime(2025, 10, 7, 10, 8, 1, 628473, tzinfo=datetime.timezone.utc), content='Chhatrapati Shivaji Maharaj, the Maratha warrior king, fought bravely against Mughal rule, liberating his people and establishing an independent Hindu kingdom.  \n', type='TextMessage')], stop_reason='External termination requested')

# Aborting a Team

it will immediate stop a team and gives cancellederror exception.

In [15]:
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console
cancel_token = CancellationToken()

run2 = asyncio.current_task(
    Console(team.run_stream(task = "Give a Short story of a lion in 30 words",cancellation_token=cancel_token))
)

await asyncio.sleep(2)
cancel_token.cancel()

try: 
    result = await run2
except:
    print("Cancelled error")

C:\Users\princ\AppData\Local\Temp\ipykernel_11652\2357897732.py:5: RuntimeWarning: coroutine 'Console' was never awaited
  run2 = asyncio.current_task(


Cancelled error
